In [ ]:
!pip install wandb -qqq
!apt install tree

     |████████████████████████████████| 1.9 MB 3.8 MB/s 
     |████████████████████████████████| 184 kB 90.6 MB/s 
     |████████████████████████████████| 174 kB 5.1 MB/s 
     |████████████████████████████████| 62 kB 1.4 MB/s 
     |████████████████████████████████| 173 kB 79.6 MB/s 
     |████████████████████████████████| 168 kB 73.4 MB/s 
     |████████████████████████████████| 168 kB 70.2 MB/s 
     |████████████████████████████████| 166 kB 52.1 MB/s 
     |████████████████████████████████| 166 kB 85.4 MB/s 
     |████████████████████████████████| 162 kB 68.1 MB/s 
     |████████████████████████████████| 162 kB 69.1 MB/s 
     |████████████████████████████████| 158 kB 66.6 MB/s 
     |████████████████████████████████| 157 kB 86.1 MB/s 
     |████████████████████████████████| 157 kB 60.5 MB/s 
     |████████████████████████████████| 157 kB 40.0 MB/s 
     |████████████████████████████████| 157 kB 79.1 MB/s 
     |████████████████████████████████| 157 kB 86.7 MB/s 
     |████████████

In [ ]:
# !wandb login --relogin

In [ ]:
import wandb
wandb.login()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
import random
import torch
import torchvision
from torch.utils.data import TensorDataset
from tqdm.auto import tqdm
import pandas as pd
import numpy as np
from torch.autograd import Variable
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
from math import sqrt
import math, time
from sklearn.metrics import mean_squared_error,mean_absolute_percentage_error
from torch.utils.data import DataLoader

from itertools import cycle
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

In [ ]:
#this block was only used to connect the google drive for colab implementations
#if you're working on your local, please skip this part

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#please add sys path to reach the modules of the functions

import sys
sys.path.append('/content/gdrive/MyDrive/Colab Notebooks/DataCraft_notebook/Code files/One py for each application/')

## FUNCTIONS

In [ ]:
#import cerated early stopping, LSTM model and sequence functions in py files

from earlyStopping import EarlyStopping
from LSTM_model import LSTM
from sequenceDataset import SequenceDataset

In [ ]:
def split(df, SeqLen): #splitdataset
  split = [int(len(df)*0.8), int(len(df)*0.9)] #give split index for train, valid, test
  train = df[:split[0]]  #~%80
  val = df[split[0]-SeqLen:split[1]] # ~%10
  test = df[split[1]-SeqLen:]  # ~%10
  return train, val, test

In [ ]:
#create evaluation function to get the MAPE and MSE scores of the train, test, validation dataset

def evaluation(split,scaled,Steps,model,loader,criterion,scaler):
  #calculate test loss, MSE and MAPE   
  loss=0    
  y_pred = torch.tensor([]).cuda()
  y=torch.tensor([]).cuda()
  y=torch.reshape(y, (-1,))
  y_pred=torch.reshape(y_pred, (-1,))
  model.eval()
  with torch.no_grad():
      for X, Y in loader:
          X=X.cuda()
          Y=Y.cuda()
          y = torch.cat((y, Y), 0)
          predict = model(X)
          
          predict=torch.reshape(predict, (-1,))
          y_pred = torch.cat((y_pred.cuda(), predict), 0)
          loss += criterion(y_pred, y).item()

  avg_loss = loss / Steps
  print("{} loss:".format(split),avg_loss)

  #reshape, concatenate, and apply inverse scaler transform to calculate the error values
  scaled_open=np.reshape(scaled[:,1], (-1,1))
  y_pred=np.reshape(y_pred.cpu().detach().numpy(), (-1,1))
  pred_open=np.concatenate((y_pred, scaled_open), axis=1)
  pred_open = scaler.inverse_transform(pred_open)
  price_open=np.concatenate((np.reshape(y.cpu().detach().numpy(), (-1,1)), scaled_open), axis=1)
  price_open = scaler.inverse_transform(price_open)

  mse_score = mean_squared_error(price_open[:,0], pred_open[:,0])
  print(split+' MSE: %.2f MSE' % (mse_score))

  mape_score = mean_absolute_percentage_error(price_open[:,0], pred_open[:,0])
  print(split+" MAPE:",mape_score)

  return (mape_score,mse_score)


## DATASET

The code user should indicate which dataset will be processed in this notebook. Because there are 3 different datasets belonging to the 3 different companies. These include the stock prices of Apple, Samsung, and Xiaomi companies.

In [ ]:
#initialize the variable
company = "Apple"

In [ ]:
#get the datset
with wandb.init(project="DSS") as r:
         
        # ✔️ declare which artifact we'll be using
        if company == 'Apple':
          artifact = r.use_artifact('metu_datacraft/DSS/Apple:v2', type='Data')
        elif company == 'Samsung':
          artifact = r.use_artifact('metu_datacraft/DSS/Samsung:v2', type='Data')
        elif company == 'Xiaomi':
          artifact = r.use_artifact('metu_datacraft/DSS/Xiaomi:v2', type='Data')
        else:
          raise Exception("Sorry, please enter the company name correctly")

        table = artifact.get('weekday_data')
        dataset= {"Date": table.get_column("Date"),"Price":table.get_column("Price"),"Open":table.get_column("Open"),
                  "High":table.get_column("High"), "Low":table.get_column("Low") , "Volume": table.get_column("Volume"),
                  "Change":table.get_column("Change")}
        data = pd.DataFrame(dataset)

wandb: Currently logged in as: fatmaoztel (metu_datacraft). Use `wandb login --relogin` to force relogin


wandb:   1 of 1 files downloaded.  


For only Apple dataset, the dataset had to be filtered to get high performance on the model prediction. This filtering issue was not applied on the other datasets belong to Samsung and Xiaomi. 

In [ ]:
#the data was filtered to get the values of stock prices after the year 2020 to get more powerful model
#this block is valid only for Apple dataset
if company == 'Apple':
  data['Date'] = pd.to_datetime(data.Date)
  data = data[data['Date']>'2020-01-01']
  data.reset_index(inplace=True)
  data.drop(columns={'index'},inplace=True)

## HYPERPARAMETER TUNING FOR ARCHITECTURE

In this section, the LSTM model was used as the main model. Firstly hyperparameter tuning was applied only to determine the number of hidden layers and nodes. The aim of this tuning was to create an architecture of the LSTM model.


In [ ]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
#create sweep config for the architecture tuning
#only use number of layer and node to tune
sweep_config = {
    'method': 'bayes',

    'metric': {
      'name': 'average_train_loss',
      'goal': 'minimize'   
    },
    'metric': {
      'name': 'average_valid_loss',
      'goal': 'minimize'   
    },
    "parameters" : {'hidden_dim': {'values': [1,2,4,8,16,32]},                      
                            'num_layers':{'values':[1,2,3]}
                    }
                }

In [ ]:
#create sweep id
sweep_id = wandb.sweep(sweep_config, project="DSS", entity="metu_datacraft")

Create sweep with ID: yg2h3a3u
Sweep URL: https://wandb.ai/metu_datacraft/DSS/sweeps/yg2h3a3u


In [ ]:
#give constant parameters for this architecture tuning
#they keep same to tune the number of layer and node
input_dim = 1
output_dim = 1
batch_size=5
n_epochs=500000
sequence_length=90

In [ ]:
def train(config=None):
  with wandb.init(config=config):
    config = wandb.config

    data['Date'] = pd.to_datetime(data.Date)

    #split dataset
    train, valid, test = split(data[["Price","Open"]],sequence_length)

    #scale the colums and values
    scaler = StandardScaler()
    train_scaled = scaler.fit_transform(train) #fit the scaler by using train dataset
    test_scaled = scaler.transform(test) #scale the test dataset
    valid_scaled = scaler.transform(valid) #scale the validation daatset

    train_data = pd.DataFrame(train_scaled, columns = ['Price','Open'])
    test_data = pd.DataFrame(test_scaled, columns = ['Price','Open'])
    valid_data = pd.DataFrame(valid_scaled, columns = ['Price','Open'])

    #get sequences of the dataset
    train_dataset = SequenceDataset(
        train_data,
        target="Price",
        features=["Open"],
        sequence_length=sequence_length
    )


    test_dataset = SequenceDataset(
        test_data,
        target="Price",
        features=["Open"],
        sequence_length=sequence_length
    )

    valid_dataset = SequenceDataset(
        valid_data,
        target="Price",
        features=["Open"],
        sequence_length=sequence_length
    )

    #create the dataloader for the dataset to create batches
    trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    testloader = DataLoader(test_dataset,batch_size=batch_size,shuffle=False)
    validloader = DataLoader(valid_dataset,batch_size=batch_size,shuffle=False)

    #call model
    model = LSTM(input_dim=input_dim, hidden_dim=config.hidden_dim, num_layers=config.num_layers,output_dim=output_dim).to(DEVICE)

    criterion = torch.nn.MSELoss()    # define criterion for loss: mean-squared error 
    optimizer = torch.optim.AdamW(model.parameters()) #with default learning rate

    #use early stopping for tuning
    early_stopping = EarlyStopping(patience=10, delta=0.0001, verbose=False)

    #keep steps to calculate average loss values
    trainSteps = len(train_dataset) //batch_size
    testSteps = len(test_dataset)//batch_size
    validSteps = len(valid_dataset)//batch_size
    count=0
    for epoch in range(n_epochs):
      model.train()
      train_loss = 0
      
      for X, y in trainloader:
          X=X.to(DEVICE)
          y=y.to(DEVICE)
          output = model(X)
          output=torch.reshape(output, (-1,))
          loss = criterion(output, y) #calculate the loss
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
          train_loss += loss.item() #calculate the total loss
            
      #check the validation loss to measure the training performance
      valid_loss=0
      model.eval()
      with torch.no_grad():
          for X, y in validloader:
              X=X.to(DEVICE)
              y=y.to(DEVICE)
              predict = model(X)
              predict=torch.reshape(predict, (-1,))
              valid_loss += criterion(predict, y).item()
              
      #calculate the loss values of train and validation        
      avg_train_loss = train_loss / trainSteps  
      avg_valid_loss = valid_loss / validSteps

      print("Epoch: %d, Train loss: %1.5f, Valid loss: %1.7f" % (epoch, avg_train_loss,avg_valid_loss))

      #log the loss values to the W&B
      wandb.log({"Epoch":epoch, "average_train_loss": avg_train_loss,"average_valid_loss":avg_valid_loss})

      early_stopping(avg_valid_loss, model)
            
      if early_stopping.early_stop:
          print("Early stopping at epoch: ", epoch)
          break

    mape_test,mse_test=evaluation("Test",test_scaled,testSteps,model,testloader,criterion,scaler)
    mape_train,mse_train=evaluation("Train",train_scaled,testSteps,model,trainloader,criterion,scaler)
    mape_valid,mse_valid=evaluation("Valid",valid_scaled,testSteps,model,validloader,criterion,scaler)

  
    #log the scores
    wandb.log({"mse_test_score": mse_test, "mape_test_score": mape_test,"mse_train_score":mse_train,"mape_train_score":mape_train})

wandb.agent(sweep_id, train)

wandb: Agent Starting Run: mji8gucd with config:
wandb: 	hidden_dim: 16
wandb: 	num_layers: 3
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch: 0, Train loss: 0.71667, Valid loss: 0.4562659
Epoch: 1, Train loss: 0.10711, Valid loss: 0.0656169
Epoch: 2, Train loss: 0.03167, Valid loss: 0.0619145
Epoch: 3, Train loss: 0.02636, Valid loss: 0.0528179
Epoch: 4, Train loss: 0.02395, Valid loss: 0.0493903
Epoch: 5, Train loss: 0.02253, Valid loss: 0.0504269
EarlyStopping counter: 1 out of 10
Epoch: 6, Train loss: 0.02069, Valid loss: 0.0539201
EarlyStopping counter: 2 out of 10
Epoch: 7, Train loss: 0.02034, Valid loss: 0.0408695
Epoch: 8, Train loss: 0.01798, Valid loss: 0.0486060
EarlyStopping counter: 1 out of 10
Epoch: 9, Train loss: 0.01685, Valid loss: 0.0367022
Epoch: 10, Train loss: 0.01676, Valid loss: 0.0305128
Epoch: 11, Train loss: 0.01530, Valid loss: 0.0307054
EarlyStopping counter: 1 out of 10
Epoch: 12, Train loss: 0.01433, Valid loss: 0.0271757
Epoch: 13, Train loss: 0.01296, Valid loss: 0.0236253
Epoch: 14, Train loss: 0.01186, Valid loss: 0.0226268
Epoch: 15, Train loss: 0.01197, Valid loss: 0.0200558
Epoch:

Epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
average_train_loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
average_valid_loss,█▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mape_test_score,▁
mape_train_score,▁
mse_test_score,▁
mse_train_score,▁
Epoch,45
average_train_loss,0.00511
average_valid_loss,0.01486
mape_test_score,0.02038


wandb: Agent Starting Run: jafemb72 with config:
wandb: 	hidden_dim: 16
wandb: 	num_layers: 3
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch: 0, Train loss: 0.46118, Valid loss: 0.0899807
Epoch: 1, Train loss: 0.02951, Valid loss: 0.0617130
Epoch: 2, Train loss: 0.02190, Valid loss: 0.0476692
Epoch: 3, Train loss: 0.02101, Valid loss: 0.0459809
Epoch: 4, Train loss: 0.02085, Valid loss: 0.0437799
Epoch: 5, Train loss: 0.01783, Valid loss: 0.0428992
Epoch: 6, Train loss: 0.01637, Valid loss: 0.0368571
Epoch: 7, Train loss: 0.01511, Valid loss: 0.0382934
EarlyStopping counter: 1 out of 10
Epoch: 8, Train loss: 0.01521, Valid loss: 0.0311883
Epoch: 9, Train loss: 0.01391, Valid loss: 0.0295331
Epoch: 10, Train loss: 0.01411, Valid loss: 0.0253140
Epoch: 11, Train loss: 0.01252, Valid loss: 0.0273580
EarlyStopping counter: 1 out of 10
Epoch: 12, Train loss: 0.01220, Valid loss: 0.0223158
Epoch: 13, Train loss: 0.01063, Valid loss: 0.0205643
Epoch: 14, Train loss: 0.01016, Valid loss: 0.0186271
Epoch: 15, Train loss: 0.01027, Valid loss: 0.0205353
EarlyStopping counter: 1 out of 10
Epoch: 16, Train loss: 0.00952, Valid los

Epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
average_train_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
average_valid_loss,█▆▄▄▄▃▄▃▂▃▂▂▂▂▂▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mape_test_score,▁
mape_train_score,▁
mse_test_score,▁
mse_train_score,▁
Epoch,51
average_train_loss,0.00487
average_valid_loss,0.00894
mape_test_score,0.0151


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: w5pv7oet with config:
wandb: 	hidden_dim: 16
wandb: 	num_layers: 3
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch: 0, Train loss: 0.59136, Valid loss: 0.1760133
Epoch: 1, Train loss: 0.04718, Valid loss: 0.0933608
Epoch: 2, Train loss: 0.02819, Valid loss: 0.0751333
Epoch: 3, Train loss: 0.02568, Valid loss: 0.0716542
Epoch: 4, Train loss: 0.02370, Valid loss: 0.0536869
Epoch: 5, Train loss: 0.02349, Valid loss: 0.0458241
Epoch: 6, Train loss: 0.02175, Valid loss: 0.0533330
EarlyStopping counter: 1 out of 10
Epoch: 7, Train loss: 0.01855, Valid loss: 0.0506147
EarlyStopping counter: 2 out of 10
Epoch: 8, Train loss: 0.01778, Valid loss: 0.0441791
Epoch: 9, Train loss: 0.01534, Valid loss: 0.0343096
Epoch: 10, Train loss: 0.01615, Valid loss: 0.0332069
Epoch: 11, Train loss: 0.01356, Valid loss: 0.0274431
Epoch: 12, Train loss: 0.01352, Valid loss: 0.0308539
EarlyStopping counter: 1 out of 10
Epoch: 13, Train loss: 0.01255, Valid loss: 0.0354975
EarlyStopping counter: 2 out of 10
Epoch: 14, Train loss: 0.01140, Valid loss: 0.0189282
Epoch: 15, Train loss: 0.00995, Valid loss: 0.0175041
Epoch:

Epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇██
average_train_loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
average_valid_loss,█▅▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mape_test_score,▁
mape_train_score,▁
mse_test_score,▁
mse_train_score,▁
Epoch,42
average_train_loss,0.00461
average_valid_loss,0.01045
mape_test_score,0.01608


wandb: Agent Starting Run: 31y1pusa with config:
wandb: 	hidden_dim: 4
wandb: 	num_layers: 1
wandb: Ctrl + C detected. Stopping sweep.
